## Import

In [26]:
import pandas as pd
import xlrd
import numpy as np

## File name and experiment set-up

In [48]:
file = r'data\MOI 0.01 drug screen 2.xlsx'
n_plates = 7
n_columns = 12
n_repeats = 3
n_concentrations = 8
start_concentration = 50

## Read data from excel file into nice dictionary of dataframes

In [65]:
book = xlrd.open_workbook(filename=file)
sheet = book.sheet_by_index(0)
row_titles = sheet.col_values(0)
start_identifier = 'Plate:'
end_identifier = '~End'

concentrations = []
concentrations.append(start_concentration)
for i in range(1, n_concentrations):
    concentrations.append(concentrations[i-1]/2)

start_indices = []
end_indices = []

for i in range(len(row_titles)):
    row_title = row_titles[i]
    if row_title==start_identifier:
        start_indices.append(i)
    elif row_title==end_identifier:
        end_indices.append(i)

data = {}

for plate in range(n_plates):
    vals = []
    start = start_indices[plate]+2
    end = end_indices[plate]-1
    
    for col in range(2, 2+n_columns):
        cells = sheet.col_slice(col, start, end)
        for cell in cells:
            vals.append(cell.value)
            

    vals = np.reshape(vals, (n_columns, end-start)).T
    df = pd.DataFrame(data=vals)
    for i in range(n_concentrations):
        df.rename(index={i:concentrations[i]}, inplace=True)
    data[f'Plate {plate+1}'] = df

## Do triplicate averaging

In [92]:
data_averaged = {}

n_samples = int(n_columns/n_repeats)

for plate in range(n_plates):
    _plate = plate+1
    df = pd.DataFrame()
    
    for n in range(n_samples):
        i = n*n_repeats
        j = (n+1)*n_repeats-1
        av_col = data[f'Plate {_plate}'].loc[:,i:j].T.mean()
        df[n] = av_col
        
    data_averaged[f'Plate {_plate}'] = df

## For each plate, divide through by final column (DMSO)

In [123]:
data_dmso_norm = {}

for plate in range(n_plates):
    df = data_averaged[f'Plate {plate+1}']
    data_dmso_norm[f'Plate {plate+1}'] = df.iloc[:,:].div(df[3], axis=0)


In [124]:
data_dmso_norm

{'Plate 1':                   0         1         2    3
 50.000000  0.009497  1.894879  1.127979  1.0
 25.000000  1.512653  1.409406  1.776586  1.0
 12.500000  2.121985  1.515805  1.991296  1.0
 6.250000   1.240623  1.108022  1.690663  1.0
 3.125000   0.827656  0.895611  1.220886  1.0
 1.562500   0.907916  0.966115  1.118834  1.0
 0.781250   0.885305  0.940592  1.114223  1.0
 0.390625   0.956920  1.008575  1.176975  1.0,
 'Plate 2':                   0         1         2    3
 50.000000  0.003021  0.003405  1.116282  1.0
 25.000000  0.545464  1.432116  1.878885  1.0
 12.500000  1.458489  2.130265  1.828639  1.0
 6.250000   1.693404  1.426784  1.503221  1.0
 3.125000   1.207061  1.184510  1.394173  1.0
 1.562500   1.007491  1.086134  1.251092  1.0
 0.781250   0.923678  0.967607  1.093037  1.0
 0.390625   0.939252  1.078142  1.257334  1.0,
 'Plate 3':                   0         1         2    3
 50.000000  0.002189  0.006479  0.006960  1.0
 25.000000  1.300647  1.285639  1.441168  1.0